# <center> 计算图和数据图可视化</center>


## 计算图与数据图概述

计算图的生成是通过将模型训练过程中的每个计算节点关联后所构成的，初体验者可以通过查看计算图，掌握整个模型的计算走向结构，数据流以及控制流的信息。对于高阶的使用人员，能够通过计算图验证计算节点的输入输出是否正确，并验证整个计算过程是否符合预期。数据图展示的是数据预处理的过程，在MindInsight可视化面板中可查看数据处理的图，能够更加直观地查看数据预处理的每一个环节，并帮助提升模型性能。

接下来我们用一个图片分类的项目来体验计算图与数据图的生成与使用。
        
## 本次体验的整体流程
1. 体验模型的数据选择使用MNIST数据集，MNIST数据集整体数据量比较小，更适合体验使用。

2. 初始化一个网络，本次的体验使用LeNet网络。

3. 增加可视化功能的使用，并设定只记录计算图与数据图。

4. 加载训练数据集并进行训练，训练完成后，查看结果并保存模型文件。

5. 启用MindInsight的可视化图界面，进行训练过程的核对。

### 数据集来源

方法一

从以下网址下载，并将数据包解压后放在Jupyter的工作目录下。

- 训练数据集：{"http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz","http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"}
- 测试数据集：{"http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz","http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"}

可执行下面代码查看Jupyter的工作目录。

In [ ]:
import os
os.getcwd()

- 训练数据集放在----`Jupyter工作目录+\MNIST_Data\train\`，此时`train`文件夹内应该包含两个文件，`train-images-idx3-ubyte`和`train-labels-idx1-ubyte` 
- 测试数据集放在----`Jupyter工作目录+\MNIST_Data\test\`，此时`test`文件夹内应该包含两个文件，`t10k-images-idx3-ubyte`和`t10k-labels-idx1-ubyte`

方法二

直接执行以下代码，会自动进行训练集的下载与解压，但是整个过程根据网络好坏情况会需要花费几分钟时间。

In [ ]:
import urllib.request   
from urllib.parse import urlparse
import gzip 

def unzip_file(gzip_path):
    """unzip dataset file
    Args:
        gzip_path: dataset file path
    """
    open_file = open(gzip_path.replace('.gz',''), 'wb')
    gz_file = gzip.GzipFile(gzip_path)
    open_file.write(gz_file.read())
    gz_file.close()
    
def download_dataset():
    """Download the dataset from http://yann.lecun.com/exdb/mnist/."""
    print("******Downloading the MNIST dataset******")
    train_path = "./MNIST_Data/train/" 
    test_path = "./MNIST_Data/test/"
    train_path_check = os.path.exists(train_path)
    test_path_check = os.path.exists(test_path)
    if train_path_check == False and test_path_check == False:
        os.makedirs(train_path)
        os.makedirs(test_path)
    train_url = {"http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"}
    test_url = {"http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"}
    
    for url in train_url:
        url_parse = urlparse(url)
        """split the file name from url"""
        file_name = os.path.join(train_path,url_parse.path.split('/')[-1])
        if not os.path.exists(file_name.replace('.gz', '')):
            file = urllib.request.urlretrieve(url, file_name)
            unzipfile(file_name)
            os.remove(file_name)
            
    for url in test_url:
        url_parse = urlparse(url)
        """split the file name from url"""
        file_name = os.path.join(test_path,url_parse.path.split('/')[-1])
        if not os.path.exists(file_name.replace('.gz', '')):
            file = urllib.request.urlretrieve(url, file_name)
            unzipfile(file_name)
            os.remove(file_name)

download_dataset()

#### 数据集使用

设置正确的数据存放路径，可将数据集读取出来，并对整体数据集做预处理，让数据更能发挥模型性能。MindInsight可视化的数据图，便是显示的数据集预处理时的变化方式和顺序。

In [ ]:
import mindspore.dataset as ds
import mindspore.dataset.transforms.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.dataset.transforms.vision import Inter
from mindspore.common import dtype as mstype


def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """
    create dataset for train or test
    """
    """define dataset"""
    mnist_ds = ds.MnistDataset(data_path)

    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    """define map operations"""
    type_cast_op = C.TypeCast(mstype.int32)
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)  # Bilinear mode
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
    rescale_op = CV.Rescale(rescale, shift)
    hwc2chw_op = CV.HWC2CHW()

    """apply map operations on images"""
    mnist_ds = mnist_ds.map(input_columns="label", operations=type_cast_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=resize_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_nml_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=hwc2chw_op, num_parallel_workers=num_parallel_workers)

    """apply DatasetOps"""
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)  # 10000 as in LeNet train script
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

### 可视化操作流程

1. 准备训练脚本，在训练脚本中指定计算图的超参数信息，使用`Summary`保存到日志中，接着再运行训练脚本。

2. 启动MindInsight，启动成功后，就可以通过访问命令执行后显示的地址，查看可视化界面。

3. 访问可视化地址成功后，就可以对图界面进行查询等操作。

#### 初始化网络

1. 导入构建网络所使用的模块。

2. 构建初始化参数的函数。

3. 创建网络，在网络中设置参数。

In [ ]:
import mindspore.nn as nn
from mindspore.common.initializer import TruncatedNormal


def conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    """weight initial for conv layer"""
    weight = weight_variable()
    return nn.Conv2d(in_channels, out_channels,
                     kernel_size=kernel_size, stride=stride, padding=padding,
                     weight_init=weight, has_bias=False, pad_mode="valid")


def fc_with_initialize(input_channels, out_channels):
    """weight initial for fc layer"""
    weight = weight_variable()
    bias = weight_variable()
    return nn.Dense(input_channels, out_channels, weight, bias)


def weight_variable():
    """weight initial"""
    return TruncatedNormal(0.02)


class LeNet5(nn.Cell):
    
    def __init__(self, num_class=10, channel=1):
        super(LeNet5, self).__init__()
        self.num_class = num_class
        self.conv1 = conv(channel, 6, 5)
        self.conv2 = conv(6, 16, 5)
        self.fc1 = fc_with_initialize(16 * 5 * 5, 120)
        self.fc2 = fc_with_initialize(120, 84)
        self.fc3 = fc_with_initialize(84, self.num_class)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

#### 主程序运行

1. 首先在主函数之前调用所需要的模块，并在主函数之前使用相应接口。

2. 本次体验主要完成计算图与数据图的可视化，定义变量`specified={'collect_graph': True,'collect_dataset_graph': True}`，在`specified`字典中，键名`collect_graph`值设置为`True`，表示记录计算图；键名`collect_dataset_graph`值设置为`True`，表示记录数据图。

3. 定义完`specified`变量后，传参到`summary_collector`中，最后将`summary_collector`传参到`model`中。

至此，模型中就有了计算图与数据图的可视化功能。

In [ ]:
import mindspore.nn as nn
from mindspore import context
from mindspore.train import Model
from mindspore.nn.metrics import Accuracy
from mindspore.train.callback import SummaryCollector
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor, TimeMonitor

if __name__ == "__main__":
    device_target = "CPU"
    
    context.set_context(mode=context.GRAPH_MODE, device_target=device_target)
    download_dataset()
    ds_train = create_dataset(data_path="./MNIST_Data/train/")

    network = LeNet5()
    net_loss = nn.SoftmaxCrossEntropyWithLogits(is_grad=False, sparse=True, reduction="mean")
    net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)
    time_cb = TimeMonitor(data_size=ds_train.get_dataset_size())
    config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
    ckpoint_cb = ModelCheckpoint(prefix="checkpoint_lenet", config=config_ck)
    model = Model(network, net_loss, net_opt, metrics={"Accuracy": Accuracy()})
    specified={'collect_graph': True,'collect_dataset_graph': True}
    summary_collector = SummaryCollector(summary_dir='./summary_dir', collect_specified_data=specified, collect_freq=1, keep_default_action=False)
    
    print("============== Starting Training ==============")
    model.train(epoch=2, train_dataset=ds_train, callbacks=[time_cb, ckpoint_cb, LossMonitor(), summary_collector], dataset_sink_mode=False)

    print("============== Starting Testing ==============")
    param_dict = load_checkpoint("checkpoint_lenet-3_1875.ckpt")
    load_param_into_net(network, param_dict)
    ds_eval = create_dataset("./MNIST_Data/test/")
    acc = model.eval(ds_eval, dataset_sink_mode=False)
    print("============== {} ==============".format(acc))

### 启动MindInsight
- 启动MindInsigh服务命令：`mindinsigh start --summary-base-dir=/path/ --port=8080`；
- 执行完服务命令后，访问给出的地址，查看MindInsigh可视化结果。

![title](https://gitee.com/mindspore/docs/raw/master/tutorials/notebook/mindinsight/images/mindinsight_map.png)

### 计算图信息
- 文本选择框：输入计算图对应的路径及文件名，显示相应的计算图，便于查找文件。
- 搜索框：可以对整体计算图的节点信息进行搜索，输入完整的节点名称，回车执行搜索，如果有该名称节点，就会呈现出来，便于查找节点。
- 缩略图：展示整体计算图的缩略情况，在面板左边查看详细图结构时，在缩略图处会有定位，显示当前查看的位置在整体计算图中的定位，实时呈现部分与整体的关系。
- 节点信息：显示当前所查看节点的信息，包括名称、类型、属性、输入和输出。便于在训练结束后，核对计算正确性时查看。
- 图例：图例中包括命名空间、聚合节点、虚拟节点、算子节点、常量节点，通过不同图形来区分。

![title](https://gitee.com/mindspore/docs/raw/master/tutorials/notebook/mindinsight/images/cast_map.png)

### 数据图信息

数据图所展示的顺序与数据集使用处代码顺序对应

1. 首先是从加载数据集`mnist_ds = ds.MnistDataset(data_path)`开始，对应数据图中`MnistDataset`。

2. 在以下所示代码中，是数据预处理的一些方法，顺序与数据图中所示顺序对应。

```
type_cast_op = C.TypeCast(mstype.int32)
resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)
rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
rescale_op = CV.Rescale(rescale, shift)
hwc2chw_op = CV.HWC2CHW()
mnist_ds = mnist_ds.map(input_columns="label", operations=type_cast_op, num_parallel_workers=num_parallel_workers)
mnist_ds = mnist_ds.map(input_columns="image", operations=resize_op, num_parallel_workers=num_parallel_workers)
mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_op, num_parallel_workers=num_parallel_workers)
mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_nml_op, num_parallel_workers=num_parallel_workers)
mnist_ds = mnist_ds.map(input_columns="image", operations=hwc2chw_op, num_parallel_workers=num_parallel_workers)
```

- `TypeCast`：在数据集`create_data`函数中，使用：`TypeCase(mstype.int32)`，将数据类型转换成我们所设置的类型。
- `Resize`：在数据集`create_data`函数中，使用：`Resize(resize_height，resize_width = 32,32)`，可以将数据的高和宽做调整。
- `Rescale`：在数据集`create_data`函数中，使用：`rescale = 1.0 / 255.0`；`Rescale(rescale,shift)`，可以重新数据格式。
- `HWC2CHW`：在数据集`create_data`函数中，使用：`HWC2CHW()`，此方法可以将数据所带信息与通道结合，一并加载。


3. 前面的几个步骤是数据集的预处理顺序，后面几个步骤是模型加载数据集时要定义的参数，顺序与数据图中对应。

```
buffer_size = 10000
mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)  # 10000 as in LeNet train script
mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
mnist_ds = mnist_ds.repeat(repeat_size)
```
        
- `Shuffle`：在数据集`create_data`函数中，使用：`buffer_size = 10000`，后面数值可以支持自行设置，表示一次缓存数据的数量。
- `Batch`：在数据集`create_data`函数中，使用：`batch_size = 32`。支持自行设置，表示将整体数据集划分成小批量数据集，每一个小批次作为一个整体进行训练。
- `Repeat`：在数据集`create_data`函数中，使用：`repeat_size = 1`，支持自行设定，表示的是一次运行中要训练的次数。

![title](https://gitee.com/mindspore/docs/raw/master/tutorials/notebook/mindinsight/images/data_map.png)

### 关闭MindInsight

- 查看完成后，在命令行中可执行此命令`mindinsight stop --port=8080`，关闭MindInsight。